# COSADAMA Introduction to Data Science Study

- 참고자료: 2020년 코사다마 데이터 분석 커리큘럼(박하람), 파이썬으로 데이터 주무르기(민형기), 점프 투 파이썬(박응용), 파이썬 입문과 크롤링 기초 부트캠프(잔재미코딩)
- 교재: 139 - 166쪽

## 시카고 샌드위치 맛집 분석 

이번 주차에는 **웹 크롤링**을 활용해 **시카고 샌드위치 맛집 분석**을 해보려 합니다. 더불어 네이버 영화 사이트에서 **네이버 영화 평점 순으로 시각화**하는 것까지 해보려고 해요.

시카고 샌드위치 맛집 분석에서는 샌드위치 맛집의 이름, 샌드위치 이름, 가게의 주소 등의 정보를 얻고, 주소로 Google Maps를 이용해 위도와 경도를 찾아 Folium으로 시각화하는 방법까지 해볼 겁니다. 더불어 웹 크롤링을 하여 정보를 가져올 때에는 꽤 시간이 걸리기 때문에 걸리는 시간을 시각화 해 알려주는 tqdm 모듈도 사용해 볼겁니다. 정말 재미있는 부분이라 이번 주차도 신나게 공부할 수 있을 것 같습니다!🐶 그럼 이번 주도 공부 열심히 해봅시다.

### urllib과 BeautifulSoup으로 사이트에 접근하기 

urllib과 BeautifulSoup으로 시카고 샌드위치 맛집 랭킹을 알려주는 사이트를 불러와 볼게요. 여기에서는 가게이름, 가게 메인 메뉴, 각 가게 소개 페이지를 가져와 볼거에요. 우선 필요한 모듈들을 모두 import 해봅시다.


In [ ]:
import urllib.request # 403 Forbidden 오류해결
from urllib.request import urlopen
from bs4 import BeautifulSoup

- [시카고 샌드위치 맛집 사이트](https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/)

교재에서는 url을 두 부분으로 나누어 놓았어요. 후에 상대주소를 절대주소 바꾸기 위해서 이렇게 하는 것이 필요해요. (자세한 설명은 후에 상대주소/절대주소를 나눌 때 설명할게요.) 이렇게 하고 html을 읽어와 보기로 해요.

In [ ]:
url_base = 'https://www.chicagomag.com'
url_sub = '/Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/'
url = url_base + url_sub
headers = {'User-Agent': 'Mozilla/5.0'}
res = urllib.request.Request(url, headers =headers)
html = urlopen(res).read()
soup = BeautifulSoup(html, "html.parser")
# soup

잘 불러와졌는지 확인해보는 것을 잊지 말구요. 그 다음 크롬의 개발자 도구를 확인해 우선 1위의 BLT와 다른 정보들이 담겨있는 태그와 클라스가 무엇인지 확인해봅니다. div 태그에 class는 sammy네요. find_all을 사용해 가져와볼게요.

In [ ]:
lists = soup.find_all('div', class_='sammy')
# lists

잘 불러와졌네요! 1) 샌드위치 이름 2) 샌드위치 가게 이름 3) 사이트 주소 4) 랭킹까지 모두 들어있죠? 그런데 너무 다양하게 많이 적혀져 있으니, 이름만 우선 get_text()로 모두 가져와 볼게요. for문으로 이를 출력해보겠습니다. len()을 사용해 정확히 50개가 맞는지도 살펴보구요.

In [ ]:
for list in lists[:5]: 
    print(list.get_text())


1
BLT
Old Oak Tap
Read more 


2
Fried Bologna
Au Cheval
Read more 


3
Woodland Mushroom
Xoco
Read more 


4
Roast Beef
Al’s Deli
Read more 


5
PB&L
Publican Quality Meats
Read more 



In [ ]:
len(lists)

50

잘 뽑아낸 것 같습니다. 

### 접근한 웹 페이지에서 원하는 데이터 추출, 정리하기

데이터를 가져와 dataframe에 넣어보겠습니다. div의 sammy에서 각각의 정보들을 정확히 꺼내어보죠.

In [ ]:
tmp_one = soup.find_all('div', 'sammy')[0]
type(tmp_one)

bs4.element.Tag

In [ ]:
tmp_one

<div class="sammy" style="position: relative;">
<div class="sammyRank">1</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/"><b>BLT</b><br/>
Old Oak Tap<br/>
<em>Read more</em> </a></div>
</div>

첫번째만 가져와서 생각해보면, type이 tag이기 때문에 여기에도 바로 find를 사용할 수 있어요. 여기에 링크가 들어있는 class를 크롬 개발자 도구로 보니 'sammyRank'에요. 가져와서 제대로 뽑아 봤나 볼게요.

In [ ]:
tmp_one.find(class_='sammyRank')

<div class="sammyRank">1</div>

그럼 안에 있는 정보만 가져와볼게요.

In [ ]:
tmp_one.find(class_='sammyRank').get_text() # .string도 가능

'1'

그 다음으로는 메뉴 이름과 가게 이름을 얻어볼게요. 클라스 이름이 'sammyListing'이네요. (크롬 개발자 도구로 반드시 태그를 확인해보세요!) 

In [ ]:
tmp_one.find(class_='sammyListing').get_text()

'BLT\nOld Oak Tap\nRead more '

엇, 나오긴 나왔는데 우리가 의도한대로 나오지 않았네요. 아주 조금 있다가 보기로 하고, 지금은 그 다음 url 주소를 확인해봅시다. 아래를 보면, 이건 get_text로 얻을 수 있는게 아니라 a 태그의 href에 들어있어요. 

In [ ]:
tmp_one

<div class="sammy" style="position: relative;">
<div class="sammyRank">1</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/"><b>BLT</b><br/>
Old Oak Tap<br/>
<em>Read more</em> </a></div>
</div>

그래서 가져올 때는 이렇게 아래와 같이 가져와야 합니다.

In [ ]:
tmp_one.find('a')['href']

'/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

근데 주소가 끊겨져 있는 것 같죠? 앞에 https://www.chicagomag.com 가 있어야 바로 사이트가 연결이 되어요. 이렇게 적혀있는 부분들을 '상대주소'라고 해요. 모두 적혀져 있어 사이트로 바로 연결되는 주소를 '절대주소'라고 하구요. 

다시 돌아가, 제대로 샌드위치 이름과 가게 이름을 구해 봅시다.

In [ ]:
tmp_one.find(class_='sammyListing').get_text()

'BLT\nOld Oak Tap\nRead more '

#### 정규 표현식 활용해 데이터 가공하기

구조를 보면 맨 앞에가 샌드위치 이름, 중간이 샌드위치 가게 이름이네요. 이를 분리하기 위해서 우리는 지난 시간에 배운 '정규 표현식'을 활용해 볼 겁니다.

In [ ]:
import re 

tmp_string = tmp_one.find(class_='sammyListing').get_text()

re.split('\r\n|\n', tmp_string)

['BLT', 'Old Oak Tap', 'Read more ']

정규표현식이 기억나지 않는다면, 다시 돌아가 봐야겠죠? \r\n과 \n 둘 중 하나라도 있으면 split 해줘라 라고 명령을 해줬습니다. 잘 분리를 해줬네요ㅎㅎ 리스트로 담겨져 있으니까 아래와 같이 샌드위치 이름과 가게 이름을 분리하면 됩니다.

In [ ]:
# 샌드위치 이름
re.split('\r\n|\n', tmp_string)[0]

'BLT'

In [ ]:
# 샌드위치 가게 이름
re.split('\r\n|\n', tmp_string)[1]

'Old Oak Tap'

#### 절대경로와 상대경로 

앞서 상대경로, 절대경로에 대한 설명을 잠깐 드렸는데요. 더 정확하게 이해를 하기 위해서 샌드위치 페이지에 있는 모든 url들을 가져와 볼게요.

In [ ]:
# 상대경로와 절대경로 - 그냥 검은색으로 되어 있는 부분은 상대경로, 링크로 되어 있는 것은 절대경로 
address_all = soup.find_all('div', 'sammy')
for item in address_all[:15]: 
    print(item.find('a')['href'])

/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/
/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Au-Cheval-Fried-Bologna/
/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Xoco-Woodland-Mushroom/
/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Als-Deli-Roast-Beef/
/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Publican-Quality-Meats-PB-L/
http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Hendrickx-Belgian-Bread-Crafter-Belgian-Chicken-Curry-Salad/
/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Acadia-Lobster-Roll/
/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Birchwood-Kitchen-Smoked-Salmon-Salad/
/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Cemitas-Puebla-Atomica-Cemitas/
/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Nana-Grilled-Laughing-Bird-Shrimp-and-Fried-Oyster-Po-Boy/
http://www.chicagomag.com/Chicago-Magazine/November-2012/

15줄만 뽑아 보았는데, 여기서 상대경로로 된 부분들이 많죠? 이것 이용해 바로 사이트를 열려면, 앞에 https://www.chicagomag.com 을 붙여주어야 합니다. 이렇게 묶어주기 위해서 urllib에 있는 urljoin을 사용할 거에요. 이 함수는 절대 경로라면 냅두고, 상대경로에만 앞의 주소를 붙여서 절대 경로로 전환시켜줘요.

In [ ]:
from urllib.parse import urljoin

urljoin('https://www.chicagomag.com', item.find('a')['href'])

'http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Longman-and-Eagle-Wild-Boar-Sloppy-Joe/'

그러면 이제 모든 정보들을 가져와 보겠습니다. 

In [ ]:
rank = []
main_menu = []
cafe_name = []
url_add =[]

list_soup = soup.find_all('div', 'sammy')

for item in list_soup:
    rank.append(item.find(class_='sammyRank').get_text())
    
    tmp_string = item.find(class_='sammyListing').get_text()
    
    main_menu.append(re.split(('\n|\r\n'), tmp_string)[0])
    cafe_name.append(re.split(('\n|\r\n'), tmp_string)[1])
    
    url_add.append(urljoin(url_base, item.find('a')['href']))

제대로 가져와졌는지 확인을 해보겠습니다.

In [ ]:
rank[:5]

['1', '2', '3', '4', '5']

In [ ]:
main_menu[:5]

['BLT', 'Fried Bologna', 'Woodland Mushroom', 'Roast Beef', 'PB&L']

In [ ]:
cafe_name[:5]

['Old Oak Tap', 'Au Cheval', 'Xoco', 'Al’s Deli', 'Publican Quality Meats']

In [ ]:
url_add[:5]

['https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/',
 'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Au-Cheval-Fried-Bologna/',
 'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Xoco-Woodland-Mushroom/',
 'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Als-Deli-Roast-Beef/',
 'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Publican-Quality-Meats-PB-L/']

개수도 50개가 맞는지 확인해볼게요.

In [ ]:
len(rank), len(main_menu), len(cafe_name), len(url_add)

(50, 50, 50, 50)

#### DataFrame에 정보 담기 

이렇게 웹 크롤링을 통해 구한 정보들을 모두 데이터 프레임에 담아 둡시다. 이제는 pandas의 DataFrame이 친숙하죠?ㅎㅎ 

In [ ]:
import pandas as pd 

data = {'Rank': rank, 'Menu':main_menu, 'Cafe':cafe_name, 'URL':url_add}
df = pd.DataFrame(data)
df.head()

,Rank,Menu,Cafe,URL
0,1,BLT,Old Oak Tap,https://www.chicagomag.com/Chicago-Magazine/No...
1,2,Fried Bologna,Au Cheval,https://www.chicagomag.com/Chicago-Magazine/No...
2,3,Woodland Mushroom,Xoco,https://www.chicagomag.com/Chicago-Magazine/No...
3,4,Roast Beef,Al’s Deli,https://www.chicagomag.com/Chicago-Magazine/No...
4,5,PB&L,Publican Quality Meats,https://www.chicagomag.com/Chicago-Magazine/No...


예히, 정보들이 잘 담겼네요. 우리가 원하는 칼럼 순으로 정보를 저장해주려면 아래와 같이 하면 됩니다.

In [ ]:
df = df[['Rank','Cafe','Menu','URL']]
df.head()

,Rank,Cafe,Menu,URL
0,1,Old Oak Tap,BLT,https://www.chicagomag.com/Chicago-Magazine/No...
1,2,Au Cheval,Fried Bologna,https://www.chicagomag.com/Chicago-Magazine/No...
2,3,Xoco,Woodland Mushroom,https://www.chicagomag.com/Chicago-Magazine/No...
3,4,Al’s Deli,Roast Beef,https://www.chicagomag.com/Chicago-Magazine/No...
4,5,Publican Quality Meats,PB&L,https://www.chicagomag.com/Chicago-Magazine/No...


잘 되었으니, 이를 우리 csv에다 저장해 둡니다. 이렇게 중간 저장을 잘 해놓지 않으면, 정보가 날라가 버리는 것은 당연하고, 웹 크롤링하는데 시간이 너무 많이 걸리면 다시 하기가 또 곤란해져요. 그러니 중간 저장을 습관화 해두는 것이 좋습니다. 

In [ ]:
df.to_csv('03. best_sandwiches_list_chicago.csv', sep=',', encoding='utf-8')

이렇게 무사히 저장까지 해보았습니다. 

### 다수의 웹페이지에서 정보 가져오기 

여기까지는 가게 이름, 가게의 샌드위치 이름, 정보들이 더 담겨있는 url 주소까지 모두 확인해보았습니다. 

- [Old Oak Tap BLT](https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/)

이 페이지를 살펴보면, 아래에 '가격'과 '주소'가 적혀져 있습니다. 그러니 여기에서 각각 가격과 주소를 얻어보도록 하겠습니다. 또 개발자 도구를 사용해 이 정보가 들어있는 태그를 확인해봅시다. 보니 p 태그에 addy class네요.

우선 그 전에 우리가 저장한 csv 파일을 불러와보겠습니다. 필요한 모듈들도 함께 뽑아와 보구요.

In [ ]:
from bs4 import BeautifulSoup 
from urllib.request import urlopen 
import pandas as pd

df = pd.read_csv('03. best_sandwiches_list_chicago.csv', encoding='utf-8', index_col=0)
df.head()

,Rank,Cafe,Menu,URL
0,1,Old Oak Tap,BLT,https://www.chicagomag.com/Chicago-Magazine/No...
1,2,Au Cheval,Fried Bologna,https://www.chicagomag.com/Chicago-Magazine/No...
2,3,Xoco,Woodland Mushroom,https://www.chicagomag.com/Chicago-Magazine/No...
3,4,Al’s Deli,Roast Beef,https://www.chicagomag.com/Chicago-Magazine/No...
4,5,Publican Quality Meats,PB&L,https://www.chicagomag.com/Chicago-Magazine/No...


첫번째 사이트부터 들어가보면, 아까 들어있는 태그가 p 태그에 addy class죠? 

In [ ]:
df['URL'][0]

'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0'}
res = urllib.request.Request(df['URL'][0], headers =headers)
html = urlopen(res).read()
soup = BeautifulSoup(html, "html.parser")

# soup

여기에서 가격과 주소가 들어있는 곳만 뽑아와 보면!

In [ ]:
soup.find('p', class_='addy')

<p class="addy">
<em>$10. 2109 W. Chicago Ave., 773-772-0406, <a href="http://www.theoldoaktap.com/">theoldoaktap.com</a></em></p>

In [ ]:
price_tmp = soup.find('p', class_='addy').get_text()
price_tmp

'\n$10. 2109 W. Chicago Ave., 773-772-0406, theoldoaktap.com'

이 정보를 또 쪼개봐야지요. 

In [ ]:
price_tmp.split()

['$10.', '2109', 'W.', 'Chicago', 'Ave.,', '773-772-0406,', 'theoldoaktap.com']

In [ ]:
price_tmp.split()[0]

'$10.'

가격에 .이 붙어있으니까 점을 없애보죠.

In [ ]:
price_tmp.split()[0][:-1]

'$10'

그럼 이제는 주소를 한 번 봅시다. 이것을 보면, 항상 뒤에서 두개를 제외하고까지가 주소인 것을 알 수 있습니다. (다른 주소들도 확인해보세요.)

In [ ]:
price_tmp.split()

['$10.', '2109', 'W.', 'Chicago', 'Ave.,', '773-772-0406,', 'theoldoaktap.com']

In [ ]:
# 주소만 가져오기
price_tmp.split()[1:-2]

['2109', 'W.', 'Chicago', 'Ave.,']

주소만 가져와봤는데, 이것이 다 따로따로 떨여져 있죠? 이것을 하나로 합하려면 join 함수를 사용하면 됩니다. 

- [파이썬 join 함수](https://wikidocs.net/13#join)

In [ ]:
' '.join(price_tmp.split()[1:-2])

'2109 W. Chicago Ave.,'

그럼 이제 주소는 해결이 됐고, 가격과 주소가 잘 뽑아지는지 3개만 뽑아서 확인을 해보겠습니다.

In [ ]:
price = []
address =[]

for n in df.index[:3]:
    res = urllib.request.Request(df['URL'][n], headers =headers)
    html = urlopen(res).read()
    soup_tmp = BeautifulSoup(html, "html.parser")

    gettings = soup_tmp.find('p', 'addy').get_text()
    price.append(gettings.split()[0][:-1])
    address.append(' '.join(gettings.split()[1:-2]))

In [ ]:
price

['$10', '$9', '$9.50']

In [ ]:
address

['2109 W. Chicago Ave.,', '800 W. Randolph St.,', '445 N. Clark St.,']

잘 나오네요! 

### tqdm 모듈을 사용해 50 페이지에 접근하기

tqdm은 웹 크롤링할 때 상태 진행이 어떻게 되는지 알려주는 바형태의 시각화입니다. 앞서 뽑아낼 정보들이 50페이지를 모두 확인해보면서 추출하는 것이기 때문에 이렇게 얼마나 진행되었는지 알려주는 것이 있으면, 더 편하게 정보를 얻을 수 있겠죠. 

- terminal에서 conda install -c conda-forge tqdm 해주기

아래와 같이 tqdm을 import 해주고, 앞서 한 대로 50 페이지를 모두 들어가 가격과 주소를 가져와 보도록 하죠. (코드를 그냥 쓰기 전에 먼저 코드를 어떻게 쓸 지 생각해보고 보는 것이 좋습니다)

In [ ]:
from tqdm import tqdm_notebook

price = []
address = []

for n in tqdm_notebook(df.index): 
    res = urllib.request.Request(df['URL'][n], headers =headers)
    html = urlopen(res).read()
    soup_tmp = BeautifulSoup(html, "html.parser")

    gettings = soup_tmp.find('p', 'addy').get_text()
    price.append(gettings.split()[0][:-1])
    address.append(' '.join(gettings.split()[1:-2]))

<ipython-input-42-1cc37d5c9b5c>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for n in tqdm_notebook(df.index):


정말 오래 걸리죠? 제대로 가져와봤는지 확인해봅시다. (제대로 안 됐으면 완전 짱나겠죠ㅎㅎ) 

In [ ]:
price[:10]

['$10', '$9', '$9.50', '$9.40', '$10', '$7.25', '$16', '$10', '$9', '$17']

In [ ]:
address[:30]

['2109 W. Chicago Ave.,',
 '800 W. Randolph St.,',
 '445 N. Clark St.,',
 '914 Noyes St., Evanston,',
 '825 W. Fulton Mkt.,',
 '100 E. Walton',
 '1639 S. Wabash Ave.,',
 '2211 W. North Ave.,',
 '3619 W. North Ave.,',
 '3267 S. Halsted St.,',
 '2537 N. Kedzie Blvd.,',
 'Multiple',
 '3124 N. Broadway,',
 '3455 N. Southport Ave.,',
 '2657 N. Kedzie Ave.,',
 '1120 W. Grand Ave.,',
 '1141 S. Jefferson St.,',
 '333 E. Benton Pl.,',
 '1411 N. Wells St.,',
 '1747 N. Damen Ave.,',
 '3209 W. Irving Park',
 'Multiple',
 '5347 N. Clark St.,',
 '2954 W. Irving Park Rd.,',
 'Multiple',
 '191 Skokie Valley Rd., Highland Park,',
 'Multiple',
 '1818 W. Wilson Ave.,',
 '2517 W. Division St.,',
 '218 W. Kinzie']

어, 그런데 여기 multiple이란게 있죠. 샌드위치 가게가 체인점이라 여러군데에 있나봐요. 일단 이것을 알아두시고 어떻게 처리할 지 고민을 해봅시다.

In [ ]:
len(price), len(address)

(50, 50)

와핫, 다 잘 가져온 것 같네요. (여러분들은 price와 address를 모두 확인해보세요~) 이 정보들을 데이터프레임에 새로운 컬럼에 넣어볼게요.

In [ ]:
df['Price'] = price 
df['Address'] = address

df.head()

,Rank,Cafe,Menu,URL,Price,Address
0,1,Old Oak Tap,BLT,https://www.chicagomag.com/Chicago-Magazine/No...,$10,"2109 W. Chicago Ave.,"
1,2,Au Cheval,Fried Bologna,https://www.chicagomag.com/Chicago-Magazine/No...,$9,"800 W. Randolph St.,"
2,3,Xoco,Woodland Mushroom,https://www.chicagomag.com/Chicago-Magazine/No...,$9.50,"445 N. Clark St.,"
3,4,Al’s Deli,Roast Beef,https://www.chicagomag.com/Chicago-Magazine/No...,$9.40,"914 Noyes St., Evanston,"
4,5,Publican Quality Meats,PB&L,https://www.chicagomag.com/Chicago-Magazine/No...,$10,"825 W. Fulton Mkt.,"


유후, 잘 됐구요! 깔끔하게 Rank를 index로 넣어봅시다.

In [ ]:
df.set_index('Rank', inplace=True)
df.head()

,Cafe,Menu,URL,Price,Address
Rank,,,,,
1,Old Oak Tap,BLT,https://www.chicagomag.com/Chicago-Magazine/No...,$10,"2109 W. Chicago Ave.,"
2,Au Cheval,Fried Bologna,https://www.chicagomag.com/Chicago-Magazine/No...,$9,"800 W. Randolph St.,"
3,Xoco,Woodland Mushroom,https://www.chicagomag.com/Chicago-Magazine/No...,$9.50,"445 N. Clark St.,"
4,Al’s Deli,Roast Beef,https://www.chicagomag.com/Chicago-Magazine/No...,$9.40,"914 Noyes St., Evanston,"
5,Publican Quality Meats,PB&L,https://www.chicagomag.com/Chicago-Magazine/No...,$10,"825 W. Fulton Mkt.,"


자, 또 우리의 정신 건강을 위해서! 중간 저장하는 것을 잊지 마세요!

In [ ]:
df.to_csv('03. best_sandwitches_list_chicago2.csv', sep=',', encoding='utf-8')

### Google Maps와 Folium으로 맛집 지도 표기 

필요한 모듈들을 다 import 하구요, 데이터도 다시 불러와봅시다.

In [ ]:
import pandas as pd
import numpy as np
import googlemaps 
import folium
from tqdm import tqdm_notebook

In [ ]:
df = pd.read_csv('03. best_sandwitches_list_chicago2.csv', index_col=0)
df.head()

,Cafe,Menu,URL,Price,Address
Rank,,,,,
1,Old Oak Tap,BLT,https://www.chicagomag.com/Chicago-Magazine/No...,$10,"2109 W. Chicago Ave.,"
2,Au Cheval,Fried Bologna,https://www.chicagomag.com/Chicago-Magazine/No...,$9,"800 W. Randolph St.,"
3,Xoco,Woodland Mushroom,https://www.chicagomag.com/Chicago-Magazine/No...,$9.50,"445 N. Clark St.,"
4,Al’s Deli,Roast Beef,https://www.chicagomag.com/Chicago-Magazine/No...,$9.40,"914 Noyes St., Evanston,"
5,Publican Quality Meats,PB&L,https://www.chicagomag.com/Chicago-Magazine/No...,$10,"825 W. Fulton Mkt.,"


구글 맵스를 활용해 경도와 위도 정보를 얻어야 지도에 시각화할 수 있겠죠. 지난 번에 얻었던 구글 키를 다시 활용하여 정보를 얻어봅시다.

In [ ]:
gmaps_key = '********************************'
gmaps = googlemaps.Client(key=gmaps_key)

아래 주소는 df의 주소에 시카고라는 주소를 추가한 겁니다. 더 정확한 정보를 얻을 수 있겠죠.

In [ ]:
target_name = df['Address'][1] + ', ' + 'Chicago'
gmaps_output = gmaps.geocode(target_name)
gmaps_output

[{'address_components': [{'long_name': '2109',
    'short_name': '2109',
    'types': ['street_number']},
   {'long_name': 'West Chicago Avenue',
    'short_name': 'W Chicago Ave',
    'types': ['route']},
   {'long_name': 'West Town',
    'short_name': 'West Town',
    'types': ['neighborhood', 'political']},
   {'long_name': 'Chicago',
    'short_name': 'Chicago',
    'types': ['locality', 'political']},
   {'long_name': 'Cook County',
    'short_name': 'Cook County',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'Illinois',
    'short_name': 'IL',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United States',
    'short_name': 'US',
    'types': ['country', 'political']},
   {'long_name': '60622', 'short_name': '60622', 'types': ['postal_code']},
   {'long_name': '4821',
    'short_name': '4821',
    'types': ['postal_code_suffix']}],
  'formatted_address': '2109 W Chicago Ave, Chicago, IL 60622, USA',
  'geometry': {'loc

여기서 경도와 위도는 geometry라는 키에 있습니다. 원하는 정보를 제대로 가져오기 위해서 한번 봅시다.

In [ ]:
gmaps_output[0]  #리스트를 벗긴 딕셔너리 형태 

{'address_components': [{'long_name': '2109',
   'short_name': '2109',
   'types': ['street_number']},
  {'long_name': 'West Chicago Avenue',
   'short_name': 'W Chicago Ave',
   'types': ['route']},
  {'long_name': 'West Town',
   'short_name': 'West Town',
   'types': ['neighborhood', 'political']},
  {'long_name': 'Chicago',
   'short_name': 'Chicago',
   'types': ['locality', 'political']},
  {'long_name': 'Cook County',
   'short_name': 'Cook County',
   'types': ['administrative_area_level_2', 'political']},
  {'long_name': 'Illinois',
   'short_name': 'IL',
   'types': ['administrative_area_level_1', 'political']},
  {'long_name': 'United States',
   'short_name': 'US',
   'types': ['country', 'political']},
  {'long_name': '60622', 'short_name': '60622', 'types': ['postal_code']},
  {'long_name': '4821',
   'short_name': '4821',
   'types': ['postal_code_suffix']}],
 'formatted_address': '2109 W Chicago Ave, Chicago, IL 60622, USA',
 'geometry': {'location': {'lat': 41.8956049,

In [ ]:
gmaps_output[0].get('geometry')

{'location': {'lat': 41.8956049, 'lng': -87.67996149999999},
 'location_type': 'ROOFTOP',
 'viewport': {'northeast': {'lat': 41.8969538802915,
   'lng': -87.67861251970848},
  'southwest': {'lat': 41.8942559197085, 'lng': -87.6813104802915}}}

리스트를 벗기면 딕셔너리 형태가 되니, get을 이용해 경도와 위도 정보가 들어있는 곳까지 왔네요. 그럼 경도와 위도 정보는 아래와 같이 찾으면 됩니다.

In [ ]:
gmaps_output[0].get('geometry')['location']

{'lat': 41.8956049, 'lng': -87.67996149999999}

In [ ]:
gmaps_output[0].get('geometry')['location']['lat']

41.8956049

In [ ]:
gmaps_output[0].get('geometry')['location']['lng']

-87.67996149999999

그러면 이제 모두를 가져와 봅시다. 이것도 tqdm 모듈을 활용해 진행상황을 알아볼 겁니다. 그 전에 중요한 것은 우리가 가져온 주소들 중에 Multiple이라 적혀있는 주소들을 어떻게 처리할지입니다. 

여기에서는 for문을 활용해 multiple이 아닌 위도와 경도 정보를 얻기로 하고, 정보가 없는 것은 넘파이를 활용해 NaN 표시를 해줄 겁니다. (혹은 우리가 배운 예외처리를 활용하여 처리할 수도 있겠죠^^)

In [ ]:
lat = []
lng = []

for n in tqdm_notebook(df.index):
    if df['Address'][n] != 'Multiple':
        target_name = df['Address'][n] + ', ' + 'Chicago'
        gmaps_output = gmaps.geocode(target_name)
        location_output = gmaps_output[0].get('geometry')
        lat.append(location_output['location']['lat'])
        lng.append(location_output['location']['lng'])
    else: 
        lat.append(np.nan)
        lng.append(np.nan)

<ipython-input-58-7186fe42c190>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for n in tqdm_notebook(df.index):


잘 뽑아졌는지 몇개만 볼게요.

In [ ]:
lat[:15], lng[:15]

([41.8956049,
  41.8846582,
  41.8905226,
  42.0583217,
  41.8866036,
  41.9000869,
  41.8590541,
  41.9102031,
  41.9097558,
  41.8345302,
  41.9276207,
  nan,
  41.9384419,
  41.9451044,
  41.930109],
 [-87.67996149999999,
  -87.6476668,
  -87.6307834,
  -87.6837479,
  -87.64853649999999,
  -87.6253366,
  -87.6252007,
  -87.68287529999999,
  -87.7176727,
  -87.6456492,
  -87.706792,
  nan,
  -87.64464369999999,
  -87.6636812,
  -87.7072408])

잘 얻어왔으니 데이터프레임에 칼럼으로 추가해봅시다.

In [ ]:
df['lat'] = lat 
df['lng'] = lng
df[['lat','lng']].head()

,lat,lng
Rank,,
1,41.895605,-87.679961
2,41.884658,-87.647667
3,41.890523,-87.630783
4,42.058322,-87.683748
5,41.886604,-87.648536


그럼 이제 folium으로 지도에 시각화 해볼게요. 먼저는 샌드위치 집들의 위도와 경도의 평균으로 지도를 가져와보겠습니다. 그래야 우리가 원하는 샌드위치 맛집들이 한 면에 들어오겠죠.

In [ ]:
mapping = folium.Map(location=[df['lat'].mean(), df['lng'].mean()], 
                                      zoom_start=11)
folium.Marker([df['lat'].mean(), df['lng'].mean()], 
                                      popup='center').add_to(mapping)
mapping

marker를 사용해 center를 표시해주었습니다. popup은 marker 클릭 시 나오는 이름을 설정해주는 거에요.

그럼 for문을 사용해 주소가 Multiple이 아닐 경우에만 지도에 표기하도록 하겠습니다. popup으로 샌드위치 맛집의 이름도 표기해주고요.

In [ ]:
mapping = folium.Map(location=[df['lat'].mean(), df['lng'].mean()], 
                     zoom_start=11)

for n in df.index:
    if df['Address'][n] != 'Multiple':
        folium.Marker([df['lat'][n], df['lng'][n]], 
                                      popup=df['Cafe'][n]).add_to(mapping)

mapping

### 과제: 한국일보 사회 지면 5 페이지 크롤링 

저번 주차에 한국일보 1페이지를 크롤링했던 것 기억하시나요? 만약 1페이지 이상을 모두 크롤링하려면 어떻게 해야 할까요? 이번 주 과제는 **한국일보 사회면의 1쪽부터 5쪽까지 타이틀만 크롤링해 엑셀 파일에 저장**하는 것입니다. 지금까지 배운 웹크롤링 지식을 총동원하여, 아래의 조건을 만족하는 데이터를 엑셀 파일에 담아 제출해주세요!

- 조건 1. 크롤링한 데이터에 xa0, \r\n와 같이 불필요한 요소들이 있을 수 있습니다. 이를 정규표현식으로 제거해주세요.
- 조건 2. 클릭하면 기사가 나오는 url 부분도 추출해서 담아보세요. 시카고 샌드위치 프로젝트를 참고하시면 더욱 쉽게 하실 수 있을 거에요 😉


- 힌트 1. https://www.hankookilbo.com/News/Society/HC01?Page=1 여기 page에 할당된 숫자를 변경해보세요. 변경한 후 어떤 페이지가 나오는지 생각해보세요. 
- 힌트 2. https://regex101.com/ 정규표현식을 확인할 수 있는 홈페이지에서 크롤링을 통해 나온 타이틀들을 복사한뒤 직접 확인해보세요.

힌트 이외에는 여러분이 프로그래밍으로 충분히 해결할 수 있으니 고민해보세요😁

### 공부를 끝내며 

이번 주 내용도 무척 재미있었죠? 여기서 잠깐! 아직 끝나지 않았습니다 ㅎㅎ 이번 주는 프로젝트가 하나 더 있습니다. 간단하지만 그래도 배울 것이 많은 **네이버 영화 평점 순 크롤링**을 해 볼 예정이에요. 그럼 다음 교안에서 만나요! 